# Group-3 - Project

Note, must use Python 3!

Functions will scrape listing data and sold_date from webpages specified. 

Removed data for sold properties.

In [ ]:
##################################################
# Import libraries
##################################################
import pandas as pd
import re
import ssl
import urllib.request
from bs4 import BeautifulSoup as BS
from urllib.request import Request, urlopen
import numpy as np
import time

#################################################
# function defined to extract miscellaneous attributes from each listing
# - get the values 
# - clean the data
# - create a datalist
#################################################

def extract_misc_information(all_misc_information):
    misc_data_dictionary = {}
    # Iterating through the all misc information scraped from the web-page
    # and generate a data dictionary for all values extracted
    for values in all_misc_information:
        misc_list_as_key_value=list(filter(str.strip,values.get_text(separator=':').strip().split(':')))
        if(len(misc_list_as_key_value) == 2):
            key = misc_list_as_key_value[0]
            value = misc_list_as_key_value[1]
            misc_data_dictionary[key]=value
    
    # Data-key for attributes to be retrieved from the miscellaneous data-dictionary
    misc_keys_as_list = ['CONSTRUCTION STATUS','LAUNDRY','HOA AMENITIES','Appliances included','Tax assessed value','Annual tax amount','HOA AMENITIES','POOL','HOA Fees Freq','STYLE','List Date','WATER/SEWER','Class','View description','LOT DESCRIPTION']

    # Iterating through the data-key and retrieveing values from miscllaneous data-dictionary
    # If value does-not exists then adding 'None'
    misc_data_list = []
    for misc_key in misc_keys_as_list:
        if misc_key in misc_data_dictionary:
            misc_data_list.append(misc_data_dictionary.get(misc_key).replace('$','').replace(',',''))
        else:
            misc_data_list.append('None')
    return misc_data_list

#################################################
# function defined to extract zestimate information from each listing
# - get the values 
# - clean the data
#################################################

def extract_zestimate_information(zestimate_information):
    zestimate_list = []
    
    # Extracting zestimate information here
    # Index 1: Zestimate value
    # Index 2: Min - Max Zestimate range.
    # Index 3: Last 30 day price increase\decrease
    # Index 4: Next 1 year price increase\decrease
    index=0
    for values in zestimate_information:
        index=index+1
        if(index == 1) :
            zestimate_value = int(values.get_text().replace('$','').replace(',',''))
            zestimate_list.append(zestimate_value)
        elif (index == 2) :
            range = values.get_text().split(' - ')
            minimum_value = range[0].replace('$','')
            if("M" in minimum_value):
                minimum_value = float(minimum_value.replace('M',''))*1000000
            elif(isinstance(minimum_value, (float,str))):
                minimum_value = float(minimum_value.replace(',',''))
            else :
                minimum_value = float(-1)
                
            maximum_value = range[1].replace('$','')
            if("M" in maximum_value):
                maximum_value = float(maximum_value.replace('M',''))*1000000
            elif(isinstance(maximum_value, (float,str))):
                maximum_value = float(maximum_value.replace(',',''))
            else :
                maximum_value = float(-1)
            zestimate_list.append(minimum_value)
            zestimate_list.append(maximum_value)
        elif(index == 3) :
            #Data -> (Ex.+$25,421 (+2.4 %))
            price_change_last_month = values.get_text().split(' ')
            if("$" in price_change_last_month[0]):
                change_money_last_month=int(price_change_last_month[0].replace('$','').replace(',',''))   
            else:
                change_money_last_month = int(-1)
            
            if("." in price_change_last_month[1]):
                change_percent_last_month = (price_change_last_month[1].replace('(','').replace('\u200a','').replace('%','').replace(')',''))
            else:
                change_percent_last_month=float(-1)

            zestimate_list.append(change_money_last_month)
            zestimate_list.append(change_percent_last_month)
        elif(index == 4) :
            #$1,078,596 (-1.2 %)
            price_change_next_year = values.get_text().split(' ')
            
            if(("$" in price_change_next_year[0])):
                change_money_next_year = (price_change_next_year[0].replace('$','').replace(',',''))
            else:
                change_money_next_year = int(-1)
               
            
            if("." in price_change_next_year[1]):
                change_percent_next_year = (price_change_next_year[1].replace('(','').replace('\u200a','').replace('%','').replace(')',''))
            else:
                change_percent_next_year = float(-1)
            
            zestimate_list.append(change_money_next_year)
            zestimate_list.append(change_percent_next_year)

    # Filling up empty\missing data with 'None'
    while(len(zestimate_list)<7):
        zestimate_list.append('None')
    
    return zestimate_list

#################################################
# function defined to extract facts and features from each listing
# - get the values 
# - clean the data
#################################################

def extract_facts_and_features(facts_and_features):
    data_list = []
    index = 0
    # Extracting first five entries of facts and fetures
    # [Type,Year_Built,Heating,Cooling,Parking]
    for values in facts_and_features:
        index = index + 1
        if(index < 6) :
            data_list.append(values.get_text())
        else:
            break
            
    return data_list

#################################################
# function defined to extract school ratings from each listing
# - get the values 
# - clean the data
#################################################
def extract_school_ratings(school_ratings):
    school_information_list = []
    # Iterating through school rating and filling up list with relevant information
    for values in school_ratings:
        school_information_list.append(values.get_text())
    # Filling up empty\missing data with 'None'
    while(len(school_information_list)<3):
        school_information_list.append('None')
    return school_information_list

#################################################
# function defined to extract school distance from each listing
# - get the values 
# - clean the data
#################################################

def extract_school_distance(schoolDistance):
    school_information_list = []

    # Iterating through school distance and filling up list with relevant information
    # Using index to pull only first three distances
    # The data here has first school - name and then distance format hence skipping first 
    # entry always
    index = 0
    for values in schoolDistance:
        index = index + 1
        if(index%2 == 0 and index < 7):
            school_information_list.append(values.get_text().replace(' mi',''))

    # Filling up empty\missing data with 'None'
    while(len(school_information_list)<3):
        school_information_list.append('None')
    return school_information_list


#################################################
# function defined to extract data from each url listing on the main page of zillow 
# Note:
# While looping through each url from the main zillow page, 
# each url had its own response time for returning the data. 
# Due to which irregular data was fetched.
# To over come this issue we created a loop which will keep hitting the url till the data is not returned 
# and then move on to the next url

#################################################

def pull_drill_down_data(url,trial_number):
    #print(url)
    # Making the website believe that you are accessing it using a Mozilla Firefox web browser
    req = Request(url, headers = {'user-Agent':'Mozilla/5.0'})
    webpage = urlopen(req)
    webpageData=webpage.read()
    
    # Creating a BeautifulSoup object of the html page for easy extraction of data
    soup = BS(webpageData, 'html.parser')
    
    # Find all tags with some given name and attributes
    fact_and_deatures = soup.findAll('span',attrs={'class':'ds-body ds-home-fact-value'})
    zestimate_information =  soup.findAll('div',attrs={'class':'content'})
    school_ratings = soup.findAll('span',attrs={'class':'ds-schools-display-rating'})
    school_distance = soup.findAll('span',attrs={'ds-school-value ds-body-small'})
    all_misc_information = soup.findAll('span',attrs={'class':'Text-aiai24-0 IJYzV'})
    
    
    # The service sometimes does-not respond back with proper data.
    # Making a recursive call here in those situations. 
    # If after 50 tries the data is not received then putting the code to
    # sleep for about 15 seconds befor trying to retrive data again
    final_data_list = []
    if(trial_number < 50 and (len(fact_and_deatures) == 0 or len(zestimate_information) == 0 or len(school_ratings) == 0 or len(school_distance) == 0 or len(all_misc_information) == 0)):
        #print(trial_number)
        if(trial_number%50 == 0):
            time.sleep(15)
        return pull_drill_down_data(url, trial_number+1)
    else:
        final_data_list = [*extract_facts_and_features(fact_and_deatures),*extract_zestimate_information(zestimate_information),*extract_school_ratings(school_ratings),*extract_school_distance(school_distance),*extract_misc_information(all_misc_information)]
        #print(final_data_list)
        return list(final_data_list)

#################################################
# function defined to extract data from the main zillow search page
# - get the values 
# - clean the data
#################################################
def pull_data(url):
    #print(url)
    # For ignoring SSL certificate errors
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE

    # Making the website believe that you are accessing it using a Mozilla Firefox web browser
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'})
    webpage = urlopen(req)
    webPageData = webpage.read()
    # Creating a BeautifulSoup object of the html page for easy extraction of data
    soup = BS(webPageData,'html.parser')

    # View how the tags are nested in the document
    # print(soup.prettify())

    # Find all tags with some given name and attributes
    listing_details = soup.findAll("div", {"class": "list-card-info"})
    listing_drilldown_url = soup.findAll('a',attrs = {'class':'list-card-link', 'tabindex':"0"}, href = True)
 

    # Retrieve the contents of a tag (list-card-info)
    listing_details_list = []
    for tag in listing_details:
        listing_details_list.append(tag.get_text(separator='/').strip().split('/'))
    #print(listing_details_list)
    
    # Retrieve links
    listing_links = []
    for div in listing_details:
        links = div.findAll('a')
        for a in links:
            listing_links.append(a['href'])

    # clean up above code
    listing_details_list_cleaned = []
    for i in listing_details_list:
        listing_details_list_cleaned.append(list(filter(str.strip,i)))
    
    listing_details_from_drilldown_cleaned = []
    for url in listing_drilldown_url:
        time.sleep(5)
        listing_details_from_drilldown_cleaned.append(pull_drill_down_data(url['href'],1))
    
    # Combining all data together
    data = [[*i, j,*k] for i,j,k in zip(listing_details_list_cleaned, listing_links,listing_details_from_drilldown_cleaned)]
    #print(data)
    return data

##################################################
def grab_webpages(num_pages, url):
    url_list = [] # create empty list for multiple pages

    # pull out first 4 pages of data
    for num in range(5, num_pages):
        text_to_replace = "{%22currentPage%22:1}"
        new_url = re.sub(text_to_replace, "{%22currentPage%22:"+str(num)+"}", url)
        url_list.append(new_url)

    # scrape data and join together
    empty_list = []  
    for url_page in url_list:
        empty_list.append(pull_data(url_page))

    # sum list of lists
    result = sum(empty_list, [])

    # create a dataframe of result
    df = pd.DataFrame(result)
    
    return df

##################################################
# call function
##################################################
# prepare dataframe
df_sold_listings = grab_webpages(19, "https://www.zillow.com/homes/recently_sold/house_type/1-_beds/3_p/?searchQueryState={%22pagination%22:{%22currentPage%22:1},%22usersSearchTerm%22:%22Bay%20Area,%20CA%22,%22mapBounds%22:{%22west%22:-122.9904853197909,%22east%22:-121.71332467525966,%22south%22:37.330427377062136,%22north%22:38.088750071545604},%22isMapVisible%22:true,%22filterState%22:{%22beds%22:{%22min%22:1},%22isForSaleByAgent%22:{%22value%22:false},%22isForSaleByOwner%22:{%22value%22:false},%22isNewConstruction%22:{%22value%22:false},%22isForSaleForeclosure%22:{%22value%22:false},%22isComingSoon%22:{%22value%22:false},%22isAuction%22:{%22value%22:false},%22isPreMarketForeclosure%22:{%22value%22:false},%22isPreMarketPreForeclosure%22:{%22value%22:false},%22isRecentlySold%22:{%22value%22:true},%22isCondo%22:{%22value%22:false},%22isMultiFamily%22:{%22value%22:false},%22isManufactured%22:{%22value%22:false},%22isLotLand%22:{%22value%22:false},%22isTownhouse%22:{%22value%22:false},%22isApartment%22:{%22value%22:false}},%22isListVisible%22:true}")

##################################################
# data cleanup and writing to relevant output files
##################################################
# drop useless columns in df
df_sold_listings = df_sold_listings.drop([1, 4, 6, 8], axis=1)

# add appropriate column headers
df_sold_listings.columns = ['address', 'price', 'bds', 'ba', 'sqft', 'link','type','year_built','heating','cooling','parking','zestimate','zestimate-Min','zestimate-Max','price-variation-last-30-days','percent-price-variation-last-30-days','price-forecast-one-year','percent-price-forecast-one-year','elementary-school-rating','junior-high-school-rating','high-school-rating','elementary-school-distance','junior-high-school-distance','high-school-distance','construction-status','laundry','hoa-ameneties','appliances-included','tax-assessed-value','annual-tax-amount','hoa-amenities','pool','hoa-fees-frequency','style','list-data','water-sewer','class','view-description','lot-description']

# clean up data for analysis
df_sold_listings['price'] = df_sold_listings['price'].str.replace('$', '')
df_sold_listings['price'] = df_sold_listings['price'].str.replace(',', '')
df_sold_listings['price'] = df_sold_listings['price'].str.replace('M', '*1e6').map(pd.eval).astype(int)
df_sold_listings['sqft'] = df_sold_listings['sqft'].str.replace(',', '')

# split address column
temp_list = [df_sold_listings['address'].str.split(',', expand=True), df_sold_listings]
temp_table = pd.concat(temp_list, axis=1)
temp_table.columns = ['street', 'city', 'zip_code', 'address', 'price', 'bds', 'ba', 'sqft', 'links','type','year_built','heating','cooling','parking','zestimate','zestimate-Min','zestimate-Max','price-variation-last-30-days','percent-price-variation-last-30-days','price-forecast-one-year','percent-price-forecast-one-year','elementary-school-rating','junior-high-school-rating','high-school-rating','elementary-school-distance','junior-high-school-distance','high-school-distance','construction-status','laundry','hoa-ameneties','appliances-included','tax-assessed-value','annual-tax-amount','hoa-amenities','pool','hoa-fees-frequency','style','list-data','water-sewer','class','view-description','lot-description']
temp_table.columns = temp_table.columns.str.strip()
df_sold_listings = temp_table

# change datatype
cols = ['price', 'sqft']
df_sold_listings[cols] = df_sold_listings[cols].apply(pd.to_numeric, errors='coerce', axis=1)


##################################################
# export to excel
##################################################
# display data
display(df_sold_listings)
fileName="output_zillow.xlsx"
df_sold_listings.to_excel(fileName,sheet_name='Sheet_name_1')  
